# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [22]:
# Import necessary packages
import sqlite3
import pandas as pd   


In [23]:
# Create the database school.sqlite 
conn = sqlite3.connect('school.sqlite')
cursor = conn.cursor()


## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [24]:
# Your code here
cursor.execute(''' 
CREATE TABLE IF NOT EXISTS contactInfo (
person_id INTEGER PRIMARY KEY AUTOINCREMENT,
first_name VARCHAR(30) NOT NULL,
last_name VARCHAR(30) NOT NULL,
role TEXT CHECK(role IN ('student', 'staff')) NOT NULL,
telephone_no VARCHAR(15) NOT NULL,
street VARCHAR(60) NOT NULL,
city TEXT NOT NULL,
state TEXT NOT NULL,
zipcode VARCHAR(20)
);
''')

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [25]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [26]:
# Iterate over the contact list and populate the contactInfo table here
for contact in contacts:
    cursor.execute(''' 
        INSERT INTO contactInfo (first_name, last_name, role, telephone_no, street, city,
        state, zipcode) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (contact['firstName'], contact['lastName'], contact['role'], contact['telephone '], contact['street'],
              contact['city'], contact['state'], contact['zipcode ']))
    
    conn.commit()
    


**Query the Table to Ensure it is populated**

In [27]:
# Your code here 
query1 = """ 
SELECT * 
FROM contactInfo;
"""
df = pd.read_sql(query1, conn)
print(df)


    person_id   first_name last_name     role telephone_no  \
0           1    Christine    Holden    staff   2035687697   
1           2  Christopher    Warren  student   2175150957   
2           3        Linda  Jacobson    staff   4049446441   
3           4       Andrew     Stepp  student   7866419252   
4           5         Jane     Evans  student   3259909290   
5           6         Jane     Evans  student   3259909290   
6           7         Mary    Raines  student   9075772295   
7           8           Ed     Lyman  student   5179695576   
8           9    Christine    Holden    staff   2035687697   
9          10  Christopher    Warren  student   2175150957   
10         11        Linda  Jacobson    staff   4049446441   
11         12       Andrew     Stepp  student   7866419252   
12         13         Jane     Evans  student   3259909290   
13         14         Jane     Evans  student   3259909290   
14         15         Mary    Raines  student   9075772295   
15      

## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [28]:
# Your code here
conn.close()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [30]:
# Create the grades table
conn = sqlite3.connect('school.sqlite')
cursor = conn.cursor()
cursor.execute(''' 
CREATE TABLE IF NOT EXISTS grades (
userId INTEGER NOT NULL,
courseId INTEGER NOT NULL,
grade TEXT,
PRIMARY KEY (userId, courseId)
)''')

conn.commit()

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [31]:
# Find the duplicate entry
query2 = """ 
SELECT first_name, last_name, role, telephone_no, street, city, state, zipcode
FROM contactInfo
GROUP BY first_name, last_name, role, telephone_no, street, city, state, zipcode
HAVING COUNT(*) > 1;
"""
df = pd.read_sql(query2, conn)
print(df)

    first_name last_name     role telephone_no                     street  \
0       Andrew     Stepp  student   7866419252  2981 Lamberts Branch Road   
1    Christine    Holden    staff   2035687697         1672 Whitman Court   
2  Christopher    Warren  student   2175150957  1935 University Hill Road   
3           Ed     Lyman  student   5179695576              3478 Be Sreet   
4         Jane     Evans  student   3259909290        1461 Briarhill Lane   
5        Linda  Jacobson    staff   4049446441        479 Musgrave Street   
6         Mary    Raines  student   9075772295      3975 Jerry Toth Drive   

        city state zipcode  
0    Hialeah    Fl   33012  
1   Stamford    CT   06995  
2  Champaign    IL   61938  
3    Lansing    MI   48933  
4    Abilene    TX   79602  
5    Atlanta    GA   30303  
6  Ninilchik    AK   99639  


In [33]:
# Delete the duplicate entry
cursor.execute (''' 
DELETE FROM contactInfo
WHERE rowid NOT IN (
SELECT MIN(rowid)
FROM contactInfo
GROUP BY first_name, last_name, role, telephone_no, street, city, state, zipcode
);
''')
conn.commit()

In [34]:
# Check that the duplicate entry was removed
query3 = """ SELECT * FROM contactInfo;"""
df = pd.read_sql(query3, conn)
print(df)

   person_id   first_name last_name     role telephone_no  \
0          1    Christine    Holden    staff   2035687697   
1          2  Christopher    Warren  student   2175150957   
2          3        Linda  Jacobson    staff   4049446441   
3          4       Andrew     Stepp  student   7866419252   
4          5         Jane     Evans  student   3259909290   
5          7         Mary    Raines  student   9075772295   
6          8           Ed     Lyman  student   5179695576   

                      street       city state zipcode  
0         1672 Whitman Court   Stamford    CT   06995  
1  1935 University Hill Road  Champaign    IL   61938  
2        479 Musgrave Street    Atlanta    GA   30303  
3  2981 Lamberts Branch Road    Hialeah    Fl   33012  
4        1461 Briarhill Lane    Abilene    TX   79602  
5      3975 Jerry Toth Drive  Ninilchik    AK   99639  
6              3478 Be Sreet    Lansing    MI   48933  


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [35]:
# Update Ed's address
cursor.execute (''' 
UPDATE contactInfo 
SET street = '2910 Simpson Avenue', city = 'York', state = 'PA', zipcode = '17403' 
WHERE first_name = 'Ed' AND last_name = 'Lyman';
''')
conn.commit()


In [37]:
# Query the database to ensure the change was made
query4 = """ SELECT * FROM contactInfo;"""
df = pd.read_sql(query4, conn)
print(df)

   person_id   first_name last_name     role telephone_no  \
0          1    Christine    Holden    staff   2035687697   
1          2  Christopher    Warren  student   2175150957   
2          3        Linda  Jacobson    staff   4049446441   
3          4       Andrew     Stepp  student   7866419252   
4          5         Jane     Evans  student   3259909290   
5          7         Mary    Raines  student   9075772295   
6          8           Ed     Lyman  student   5179695576   

                      street       city state zipcode  
0         1672 Whitman Court   Stamford    CT   06995  
1  1935 University Hill Road  Champaign    IL   61938  
2        479 Musgrave Street    Atlanta    GA   30303  
3  2981 Lamberts Branch Road    Hialeah    Fl   33012  
4        1461 Briarhill Lane    Abilene    TX   79602  
5      3975 Jerry Toth Drive  Ninilchik    AK   99639  
6        2910 Simpson Avenue       York    PA   17403  


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [38]:
# Your code here
conn.commit()
conn.close()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 